# Initial Setup

In [0]:
import tensorflow as tf
print(tf.__version__)

In [0]:
!pip install wandb
import wandb
wandb.login()

In [0]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from wandb.keras import WandbCallback
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import time
import cv2
from tqdm.notebook import tqdm
from imutils import paths
tf.random.set_seed(666)
np.random.seed(666)

tfds.disable_progress_bar()

# Imagenet Subset 

In [0]:
!git clone https://github.com/thunderInfy/imagenet-5-categories


fatal: destination path 'imagenet-5-categories' already exists and is not an empty directory.


In [0]:
# Train and test image paths
train_images = list(paths.list_images("imagenet-5-categories/train"))
test_images = list(paths.list_images("imagenet-5-categories/test"))


In [0]:
def prepare_images(image_paths):
    images = []
    labels = []

    for image in tqdm(image_paths):
        image_pixels = plt.imread(image)
        image_pixels = cv2.resize(image_pixels, (128,128))
        image_pixels = image_pixels/255.

        label = image.split("/")[2].split("_")[0]

        images.append(image_pixels)
        labels.append(label)

    images = np.array(images)
    labels = np.array(labels)

    print(images.shape, labels.shape)

    return images, labels

In [0]:
X_train, y_train = prepare_images(train_images)
X_test, y_test = prepare_images(test_images)


(1250, 128, 128, 3) (1250,)



(250, 128, 128, 3) (250,)


In [0]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_test_enc = le.transform(y_test)


In [0]:
train_ds=tf.data.Dataset.from_tensor_slices((X_train,y_train_enc))
validation_ds=tf.data.Dataset.from_tensor_slices((X_test,y_test_enc))

In [0]:
@tf.function
def aug(image, label):
  x=tf.image.random_brightness(image,max_delta=0)
  x=tf.image.random_contrast(x,lower=0.2, upper=1.8)
  x = tf.image.random_saturation(x, lower=0.2, upper=1.5)
  x = tf.image.random_hue(x, max_delta=0.4)
  x = tf.clip_by_value(x, 0, 1)

  return x, label

In [0]:
IMG_SHAPE = 128
BS = 64
AUTO = tf.data.experimental.AUTOTUNE
train_ds = (
    train_ds
    .shuffle(100)
    .batch(BS)
    .map(aug, num_parallel_calls=AUTO)
    .prefetch(AUTO)
)
validation_ds = (
    validation_ds
    .shuffle(100)
    .batch(BS)
    .prefetch(AUTO)
)

# Model building and training wih RMSprop


In [0]:
resnet50 = tf.keras.applications.ResNet50(weights=None, include_top=False)
model = tf.keras.Sequential([resnet50,GlobalAveragePooling2D(),Dropout(0.25),Dense(5,activation='softmax')])

In [0]:
decay_steps = 1000
lr_decayed_fn = tf.keras.experimental.CosineDecay(
    initial_learning_rate=0.001, decay_steps=decay_steps)

model.compile(optimizer=tf.keras.optimizers.RMSprop(lr_decayed_fn),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [0]:
es = tf.keras.callbacks.EarlyStopping(monitor="val_sparse_categorical_accuracy", patience=2,
	restore_best_weights=True, verbose=2)

In [0]:
import time
import wandb

wandb.init(entity='authors',project='scl')
start = time.time()
model.fit(train_ds,
          validation_data=validation_ds,
          epochs=50,
          callbacks=[wandb.keras.WandbCallback(), es])
end = time.time()
wandb.log({"training_time": end - start})

Epoch 1/50
20/20 [==============================] - 6s 312ms/step - loss: 1.6051 - sparse_categorical_accuracy: 0.2976 - val_loss: 1.7048 - val_sparse_categorical_accuracy: 0.2000
Epoch 2/50
20/20 [==============================] - 5s 263ms/step - loss: 1.5929 - sparse_categorical_accuracy: 0.3056 - val_loss: 1.7039 - val_sparse_categorical_accuracy: 0.2000
Epoch 3/50
20/20 [==============================] - 5s 271ms/step - loss: 1.6160 - sparse_categorical_accuracy: 0.2792 - val_loss: 1.6867 - val_sparse_categorical_accuracy: 0.2120
Epoch 4/50
20/20 [==============================] - 4s 223ms/step - loss: 1.5946 - sparse_categorical_accuracy: 0.2776 - val_loss: 1.6938 - val_sparse_categorical_accuracy: 0.2000
Epoch 5/50
20/20 [==============================] - 5s 273ms/step - loss: 1.5765 - sparse_categorical_accuracy: 0.2880 - val_loss: 1.5870 - val_sparse_categorical_accuracy: 0.3040
Epoch 6/50
20/20 [==============================] - 6s 276ms/step - loss: 1.5180 - sparse_categorica

In [0]:
model.save_weights("full_supervised_learning.h5")

In [0]:
wandb.save("full_supervised_learning.h5")

['/content/wandb/run-20200528_111108-2h40mbhd/full_supervised_learning.h5']